In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [5]:
# Install these packages with these specific versions else the notebook breaks
!pip install transformers==4.5.1
!pip install pytorch_lightning==1.2.10
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 6.4MB/s 
     |████████████████████████████████| 901kB 22.2MB/s 
     |████████████████████████████████| 3.3MB 39.2MB/s 
     |████████████████████████████████| 849kB 6.7MB/s 
     |████████████████████████████████| 829kB 10.5MB/s 
     |████████████████████████████████| 112kB 22.1MB/s 
     |████████████████████████████████| 276kB 16.7MB/s 
     |████████████████████████████████| 184kB 21.9MB/s 
     |████████████████████████████████| 1.3MB 18.7MB/s 
     |████████████████████████████████| 143kB 38.1MB/s 
     |████████████████████████████████| 296kB 34.8MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=4c7fd9ff9a2cf35e63e60e2b2339bf08a1cb5aaae454c35315ce1714fbc93bc1
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=224ba216e2f5e5b2da16951ea2acf5

In [6]:
# Import packages
import argparse
import glob
import pickle
import os
import json
import time
import logging
import random
import re
from tqdm import tqdm
from itertools import chain
from string import punctuation
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import sentencepiece


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
import csv
from dataclasses import dataclass

from enum import Enum
from typing import List, Optional
from transformers import PreTrainedTokenizer

In [8]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [9]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "val_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [10]:
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    
    if type(hparams) is dict: 
      hparams = argparse.Namespace(**hparams)
    
    self.hparams = hparams
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)
  
  def is_logger(self):
    return self.trainer.global_rank <= 0
  
  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]
    
    return loss

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.log('training_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"train_loss": loss}
    return {"loss": loss, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    self.log('avg_training_loss', avg_train_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"avg_train_loss": avg_train_loss}
    return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"val_loss": loss}
    return {"val_loss": loss, "log": tensorboard_logs}
  
  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    print(avg_loss)
    tensorboard_logs = {"val_loss": avg_loss}
    self.log('avg_val_loss', avg_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self,
                     epoch=None, 
                     batch_idx=None, 
                     optimizer=None, 
                     optimizer_idx=None, 
                     optimizer_closure=None, 
                     on_tpu=None, 
                     using_native_amp=None, 
                     using_lbfgs=None
                     ):

    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [11]:
# The below code is adapted from:
# https://github.com/huggingface/transformers/blob/master/examples/multiple-choice/utils_multiple_choice.py

@dataclass(frozen=True)
class InputExample:
    """
    A single training/test example for multiple choice
    Args:
        example_id: Unique id for the example.
        question: string. The untokenized text of the second sequence (question).
        contexts: list of str. The untokenized text of the first sequence (context of corresponding question).
        endings: list of str. multiple choice's options. Its length must be equal to contexts' length.
        label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """

    name: str
    ingredients: str
    steps: List[str]
    label: Optional[str]

class Split(Enum):
    train = "train"
    dev = "dev"
    test = "test"

class DataProcessor:
    """Base class for data converters for multiple choice data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

class RecipeProcessor(DataProcessor):
    """Processor for the SWAG data set."""

    def get_train_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} train".format(data_dir))
        return self._create_examples(self._read_csv(os.path.join(data_dir, "train.csv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} dev".format(data_dir))
        return self._create_examples(self._read_csv(os.path.join(data_dir, "val.csv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} dev".format(data_dir))
        raise ValueError(
            "For swag testing, the input file does not contain a label column. It can not be tested in current code"
            "setting!"
        )
        return self._create_examples(self._read_csv(os.path.join(data_dir, "test.csv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1", "2", "3"]

    def _read_csv(self, input_file):
        with open(input_file, "r", encoding="utf-8") as f:
            return list(csv.reader(f))

    def _create_examples(self, lines: List[List[str]], type: str):
        """Creates examples for the training and dev sets."""

        examples = [
            InputExample(
                name=line[1],
                # common beginning of each
                # choice is stored in "sent2".
                ingredients=eval(line[3]),
                steps=eval(line[2]),
                label=eval(line[2]),
            )
            for line in lines[1:]  # we skip the line with the column names
        ]

        return examples

In [12]:
class RecipeDataset(Dataset):
    
    def __init__(self, tokenizer, data_dir, type_path,  max_len=512, mask_percent=0.4, mode='fill'):
        self.data_dir = data_dir
        self.type_path = type_path
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.ingredients = []
        self.names = []
        self.mode = mode
        self.mask_percent = mask_percent
        
        self.proc = RecipeProcessor()
        self._build()
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, 
                "target_ids": target_ids, "target_mask": target_mask}
  
    def __len__(self):
        return len(self.inputs)
  
    def _build(self):
        if self.type_path == 'train':
            examples = self.proc.get_train_examples(self.data_dir)
        else:
            examples = self.proc.get_dev_examples(self.data_dir)
    
        for i, example in enumerate(examples):
            if i % 10000 == 0:
                print(i)
            self._create_features(example)
    
    def _create_shuffle_completion(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          input_words = []
          label = []
        else:
          
          mask_words = np.round(0.5*total_words).astype(int)
          if np.random.uniform() < 0.5:
            input_words = words[:mask_words] + ['<extra_id_0>']
            label = ['<extra_id_0>'] + words[mask_words:] + ['<extra_id_1>']
          else:
            input_words = ['<extra_id_0>'] + words[mask_words:]
            label = ['<extra_id_0>'] + words[:mask_words] + ['<extra_id_1>']

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
    
    def _create_completion(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          input_words = []
          label = []
        else:
          mask_words = np.round(0.5*total_words).astype(int)
          input_words = words[:mask_words] + ['<extra_id_0>']
          label = ['<extra_id_0>'] + words[mask_words:] + ['<extra_id_1>']

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
  
    def _create_masks(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          mask_words = 0
        else:
          mask_words = np.round(self.mask_percent*total_words).astype(int)

        mask_indices = np.random.choice(np.arange(total_words), mask_words)
        
        input_words = []
        label = []
        j = 0
        
        for i, word in enumerate(words):
            if i in mask_indices:
                # Add a sentinel token in place of the word to be masked
                input_words.append('<extra_id_{}>'.format(j))
                # Add this word's token to the label list
                label.append('<extra_id_{}>'.format(j))
                # Add the corresponding label to the list
                label.append(word)
                # Step the counter by 1
                j+=1
            # If this index is not among the sampled indices 
            # just append the word
            else:
                input_words.append(word)
         
        # Add in an extra ID token like the format says
        label.append('<extra_id_{}>'.format(j))

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
    
    def _create_features(self, example):
    
        ingredients_ = ",".join(example.ingredients)
        name_ = example.name
        
        if self.mode == 'fill':
          masked_, label_ = self._create_masks(example.steps)
        
        elif self.mode == 'complete':
          masked_, label_ = self._create_completion(example.steps)

        elif self.mode == 'shuffle_complete':
          masked_, label_ = self._create_completion(example.steps)  
                                        
        input_ = "name: %s  ingredients: %s masked: %s </s>" % (name_, ingredients_, masked_)
        target =  label_ + " </s>"
        
        # tokenize inputs
        tokenized_inputs = self.tokenizer.batch_encode_plus(
            [input_], max_length = 200, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )
        # tokenize targets
        tokenized_targets = self.tokenizer.batch_encode_plus(
            [target], max_length = 150, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )

        self.inputs.append(tokenized_inputs)
        self.targets.append(tokenized_targets)
        self.ingredients.append(example.ingredients)
        self.names.append(example.name)

In [16]:
def generate_loop(checkpoint_dir='drive/MyDrive/recipe-generation/t5_fill_blanks/', 
                  checkpoint_name = 'exp_7_fb_ef.ckpt', 
                  experiment_name = 'experiment_7',
                  mode = 'fill'):
  '''
  ---------
  Generation loop
  --------
  '''
  checkpoint_path = os.path.join(checkpoint_dir, checkpoint_name)
  model = T5FineTuner.load_from_checkpoint(checkpoint_path)
  tokenizer = T5Tokenizer.from_pretrained("t5-base")

  dataset =  RecipeDataset(tokenizer, data_dir='drive/MyDrive/recipe-generation/data', type_path='val', mode=mode)
  loader = DataLoader(dataset, batch_size=32, num_workers=2)

  model.model.eval()
  outputs = []
  targets = []

  for i, batch in enumerate(tqdm(loader)):

    # Need to change max. length argument
    outs = model.model.generate(input_ids=batch['source_ids'], 
                                attention_mask=batch['source_mask'], 
                                max_length=150)
    dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
    target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

    if i%100 == 0:
      print(dec)
      print(target)

    outputs.extend(dec)
    targets.extend(target)

  results = [dataset.names, dataset.ingredients, outputs, targets]
  results_path = os.path.join(checkpoint_dir, '{}_outputs.pkl'.format(experiment_name))
  
  return(results) 

In [17]:
def read_results(results_path):
    '''
    ----------
    Load results from pkl. file
    -----------
    '''
    # Open statement within context
    with open(results_path, 'rb') as f:
      # Load statement
      results = pickle.load(f)
    # Return statement
    return(results)

In [28]:
checkpoint_dir='drive/MyDrive/recipe-generation/t5_fill_blanks/'

In [18]:
checkpoint_name = 'exp_7_fb_ef.ckpt' 
experiment_name = 'experiment_7'
mode = 'fill'

results = generate_loop(checkpoint_name=checkpoint_name, experiment_name=experiment_name, mode=mode)

0


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


10000



  0%|          | 1/568 [00:10<1:36:06, 10.17s/it]

['saucepan the cranberries with and bring the 5 minutes add to saucepan and to, minutes stir in the pomegranate and serve immediately or at', 'a add yeast lemon juice and mixture soft dough coat tin a dough cook heat 3 4 minutes is set continue 3 minutes from heat and with serve with of', 'combine is tender from add the, 1 2 cup water seasonings to', ', honey salt and vanilla blend for seconds into a minutes or until knife out clean center serve', 'heat olive saucepan heat add onion garlic saute onions translucent and saute broth sugar cayenne with and ground water a water stir 2 minutes remove and add of season', '9x13 pan with cracker together, and in bowl layer in milk over top serving squares remove', 'medium, combine and blend for saucepan rice heat mixture medium-high add and stir until preheat to a baking dish add, milk vanilla until smooth 30 soft into and bake until a, pistachios serve a / or serving', 'preheat oven 12 cups line muffin halfway baking in bowl large set in large

...


 71%|███████   | 401/568 [58:35<21:10,  7.61s/it]

['preheat to the butter and water large in together the soda, salt flour combine dry ingredients chips dough 1-inch a greased minutes or until to bake', 'preheat to baking a combine sugar butter and with an mixture meal flour, chips corn, salt a medium bowl on speed, constantly chips and mixture in the in the pan,,,,,, into store in', '2 of cut 1 inch remove the to and blender until a', 'mix dry ingredients cup a sprinkling cornstarch the ingredients bowl not baking store mixture', 'degrees and add eggs, butter and well salt chocolate nuts drop by teaspoonfuls rounded by rounded cookie sheet bake for cookies cool on a to', 'dry in 2 sugar 1 tsp add flour and remove stir sift ingredients to mixture stir moistened knead dough cover in size bowl a add well shape into buns roll dough a, cut into / / / / 2 / 2 tsp sugar-blot', 'oven to all ingredients the transfer greased 9 until top browned 30 and bake 30 carefully, onions brown', 'preheat the the of for minutes each off like, rub with spr


 88%|████████▊ | 501/568 [1:12:00<08:25,  7.54s/it]

['2 a pan capers, for or until golden and tender to a the oil pan add capsi, 3 capsi are stirring stir to a serving juice with', 'tomato, onion, garlic, paste, salt, pepper black pepper simmer medium in small combine, toss toasted, shredded cheese a are of time', 'and in a mayonnaise, and blended macaroni carrot, macaroni, coat serve!', ', combine frozen strawberries blend until smooth distributed blend smooth', 'in bowl, and and place steaks sides the, place bowl sure steaks cover refrigerate up to and lightly of lemon juice with grill 3 each side, until', '9x13 inch spray with a, salt vanilla milk vanilla until pour into prepared baking 45', 'margarine small mix the back spoon', 'layer jar jar or jar by make the with lid and shake the rim wrap, is, and a rim as, the jar and shake eggs 1 / 2 / tsp, / tsp / tsp / 2 the tsp', 'the sheet butter place a skillet a small, heavy sugar corn syrup cook heat thermometer degrees stirring nuts of vanilla mixture a smooth paste a combine & pan fro


100%|██████████| 568/568 [1:20:57<00:00,  8.55s/it]


In [24]:
len(results[3])

18147

In [29]:
results_path = os.path.join(checkpoint_dir, '{}_outputs.pkl'.format(experiment_name))

In [36]:
with open(results_path, 'wb') as f:
  pickle.dump(results, f)
  f.close()

In [31]:
results_path

'drive/MyDrive/recipe-generation/t5_fill_blanks/experiment_7_outputs.pkl'

EOFError: ignored

In [35]:
results

[['cranberry pomegranate sauce',
  'gluten free crumpets',
  'supu ya kuku  chicken soup of zanzibar',
  'miracle custard pie',
  'moroccan spiced chickpea soup',
  'hello dollies   chocolate and coconut bar cookies',
  'ground rice pudding  mehallabia',
  'pumpkin oat streusel muffins',
  'banana cinnamon bundt cake',
  'millet vegetable soup',
  'south american style steak',
  'eggplant stew  simple  persian style',
  'bavarian mint coffee',
  'couldn t be easier elephant ears',
  'green onion and potato salad',
  'watercress  radicchio   green bean salad with goat cheese for ',
  'orange lemon liqueur',
  'hot chocolate stinger',
  'good for you spicy chicken',
  'pasta with mushroom cream sauce',
  'fresh asparagus with buttered almonds',
  'bavarian liverwurst dip',
  'sweet   spicy shrimp  chambord shrimp',
  'balkenbrij  liverwurst',
  'stan s barbecue sauce',
  'my own garlic bread',
  'cornbread dressing recipe from ebony magazine',
  'chicken lemone',
  'quick  easy  yummy ap

In [37]:
    with open(results_path, 'rb') as f:
      # Load statement
      results_test = pickle.load(f)

In [39]:
len(results_test)

4